In [1]:
# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import IntProgress
from IPython.display import display

# Append base directory
import os,sys,inspect
rootname = "pub-2020-exploratory-analysis"
# thispath = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
thispath = os.getcwd()
rootpath = os.path.join(thispath[:thispath.index(rootname)], rootname)
sys.path.append(rootpath)
print("Appended root directory", rootpath)


from mesostat.utils.qt_helper import gui_fnames, gui_fpath
from mesostat.metric.metric import MetricCalculator
from mesostat.utils.hdf5_io import DataStorage
from lib.gallerosalas.data_fc_db_raw import DataFCDatabase
import lib.analysis.activity as activity
from lib.analysis.visualization import merge_image_sequence_movie

%load_ext autoreload
%autoreload 2

Appended root directory /home/alyosha/work/git/pub-2020-exploratory-analysis


In [2]:
params = {}
# params['root_path_data']  = gui_fpath("Path to data collection",  './')
params['root_path_data'] = '/media/alyosha/Data/TE_data/yasirdata_raw/'
# params['root_path_data'] = '/home/alyosha/data/yasirdata_raw/'
# params['root_path_data'] = '/media/aleksejs/DataHDD/work/data/yasir/yasirdata_raw'

In [3]:
dataDB = DataFCDatabase(params)

Reading channel label file
Reading channel area file
Reading allen brain map
Reading task structure
Reading session structure
Searching for data files
Found mice ['mou_5', 'mou_6', 'mou_7', 'mou_9']


In [4]:
ds = DataStorage('gallerosalas_result_activity.h5')

In [5]:
mc = MetricCalculator(serial=True, verbose=False)

In [6]:
print('mice', dataDB.mice)
print('nSessions', len(dataDB.sessions))
print('datatypes', dataDB.get_data_types())
print('nChannel', dataDB.get_nchannels('mou_5'))

mice {'mou_5', 'mou_9', 'mou_7', 'mou_6'}
nSessions 4
datatypes ['raw', 'bn_session', 'bn_trial']
nChannel 27


In [7]:
dataDB.calc_shortest_distances()

## Compute mean over time for all mice and condition combinations

In [8]:
exclQueryLst = [
    {'datatype' : 'bn_trial', 'intervName' : 'PRE'},  # Baseline normalized
    {'mousename' : 'mou_6', 'intervName' : 'REW'},    # No reward for this mouse
]

In [9]:
trialTypes = ['Hit', 'CR', 'Miss', 'FA']

In [ ]:
activity.compute_mean_interval(dataDB, ds, {'Hit', 'CR'}, exclQueryLst=exclQueryLst)

# 1. Significance

## 1.1 Plot Hit activity by channel

In [ ]:
activity.activity_brainplot_mousephase(dataDB, trialTypes,
                                       vmin=-0.02, vmax=0.02, drop6=True)

In [ ]:
activity.activity_brainplot_mousetrialtype(dataDB, trialTypes,
                                           vmin=-0.02, vmax=0.02, drop6=True)

In [ ]:
activity.activity_brainplot_mousephase_subpre(dataDB, trialTypes,
                                              vmin=-0.02, vmax=0.02, drop6=True)

In [ ]:
activity.activity_brainplot_mousephase_submouse(dataDB, trialTypes,
                                                vmin=-0.02, vmax=0.02, drop6=True)

## 1.2. Accuracy of hit vs cr prediction by mouse, phase

In [ ]:
activity.significance_brainplot_mousephase_byaction(dataDB, ds, performance=None,
                                                    metric='accuracy', minTrials=10,
                                                    limits=(0.5, 1.0))

In [ ]:
activity.plot_classification_accuracy_bychannel(dataDB, drop6=True)

## 1.3. Plot activity movies by chanel

In [ ]:
activity.activity_brainplot_movie_mousetrialtype(dataDB, trialTypes, vmin=-0.02, vmax=0.02,
                                                 haveDelay=True, fontsize=20)

In [11]:
merge_image_sequence_movie('activity_brainplot_mousetrialtype_bn_trial_', '.png', 0, 160,
                           trgPathName=None, deleteSrc=True)

Writing video to activity_brainplot_mousetrialtype_bn_trial_
Processing image[159]
 Done


In [12]:
merge_image_sequence_movie('activity_brainplot_mousetrialtype_bn_session_', '.png', 0, 160,
                           trgPathName=None, deleteSrc=True)

Writing video to activity_brainplot_mousetrialtype_bn_session_
Processing image[159]
 Done


# 2. Consistency

## 2.1. Consistency of channel activity

* DF/F should be independent of channel brightness/opacity, since it normalizes baseline.
* Thus, should be able to compare DFF of different mice directly

## 2.2 Channels significant for Hit/Cr. Overlap by mouse

1. Loop over datatype, interval, session
2. Test each channel for Hit/Cr
3. Count significant sessions for each channel
4. Scatter counts

**TODO**
* Split by Naive/Expert

In [ ]:
activity.plot_consistency_significant_activity_byaction(dataDB, ds)

## 2.3. Channels significant for phase. Overlap by mouse

In [ ]:
activity.plot_consistency_significant_activity_byphase(dataDB, ds, ['TEX', 'DEL'])